In [1]:
import os
from google.cloud import bigquery
import pandas as pd 


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= '/Users/jordanmaulino/Downloads/coastal-dynamo-309722-2032fe38e9cd.json'

In [3]:
client = bigquery.Client()

In [4]:
dataset_ref = client.dataset("covid19_ecdc", project="bigquery-public-data")

dataset=client.get_dataset(dataset_ref)

In [5]:
tables = list(client.list_tables(dataset))

In [6]:
for table in tables:
    print(table.table_id)

covid_19_geographic_distribution_worldwide


In [7]:
table_ref = dataset_ref.table("covid_19_geographic_distribution_worldwide")

table=client.get_table(table_ref)

In [8]:
table.schema

[SchemaField('date', 'DATE', 'NULLABLE', None, (), None),
 SchemaField('day', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('month', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('year', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('daily_confirmed_cases', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('daily_deaths', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('confirmed_cases', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('deaths', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('countries_and_territories', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('geo_id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('country_territory_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('pop_data_2019', 'INTEGER', 'NULLABLE', None, (), None)]

In [9]:
client.list_rows(table, max_results=5).to_dataframe()

,date,day,month,year,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2019
0,2020-03-20,20,3,2020,1,0,1,0,Chad,TD,TCD,15946882
1,2020-03-21,21,3,2020,0,0,1,0,Chad,TD,TCD,15946882
2,2020-03-22,22,3,2020,1,0,2,0,Chad,TD,TCD,15946882
3,2020-03-23,23,3,2020,0,0,2,0,Chad,TD,TCD,15946882
4,2020-03-24,24,3,2020,0,0,2,0,Chad,TD,TCD,15946882


In [10]:
#query to select covid cases by country on 12/1/2020
query = """
        SELECT daily_confirmed_cases, confirmed_cases, daily_deaths, countries_and_territories
        FROM `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide`
        WHERE date = '2020-12-01'
        ORDER BY daily_confirmed_cases DESC
        """


query_job = client.query(query)

covid_cases_12_1_2020 = query_job.to_dataframe()

covid_cases_12_1_2020.head()

,daily_confirmed_cases,confirmed_cases,daily_deaths,countries_and_territories
0,157903,13541224,1172,United_States_of_America
1,31219,617310,188,Turkey
2,31118,9462809,482,India
3,26338,2295654,368,Russia
4,21138,6335878,287,Brazil
